# Imports and Functions

In [ ]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from matplotlib.colors import ListedColormap, BoundaryNorm
import os
from ImagePlotDigitizer_funcs import color_to_value, generate_x_axis_map, generate_y_axis_map, crop_image
%matplotlib widget

# Load Image into an rgb array

In [ ]:
cwd = os.getcwd()
image_path = f'{cwd}/test_images/F2_McNeill.png'
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig,ax1 = subplots(1,1)
ax1.imshow(img_rgb)

# Define image colormap, edit if needed

In [ ]:
colormap = plt.colormaps['hot'].copy()
# Extract the colors from the original colormap
colors = colormap(np.linspace(0, 1, colormap.N))
# Modify the last colors to match overbound colors in image
colors[-2] = (0.4,0,0.6,1)
colors[-1] = (0.1,0,0.6,1)
# Create a new colormap with the modified colors
colormap = ListedColormap(colors)
colormap

# Map RGB image array to colormap to convert it to value only array

In [ ]:
# Extract colors and map to data values
img_vals = np.zeros((img_rgb.shape[0], img_rgb.shape[1]))
for i in range(img_rgb.shape[0]):
    for j in range(img_rgb.shape[1]):
        color = img_rgb[i, j]  # Normalize color
        value = color_to_value(color, colormap, num_samples=200,  log=False)
        img_vals[i, j] = value
fig,ax1 = subplots(1,1)
ax1.imshow(img_vals,
          cmap='turbo')

# Using interactive plot (above) fill in two x-axis values and corresponding column number

In [ ]:
x1 = 0
col1 = 96
x2 = 2
col2 = 457
x_axis = generate_x_axis_map((col1, x1), (col2, x2), img_vals)

# # Example syntax for non-linear x axis:
# x_indices = [  32, 100, 168, 211, 239, 268, 297, 328, 359]
# x_coords =  [-0.5,   0, 0.5, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8]
# x_axis = generate_arb_x_axis_map(x_indices, x_coords, img_vals)

# Repeat for y-axis values and row number

In [ ]:
y1 = 2
row1 = 22
y2 = 0
row2 = 384
y_axis = generate_y_axis_map((row1, y1), (row2, y2), img_vals)

# # Example syntax for non-linear y axis:
# y_indices = [ 16,  50,  83, 113, 143, 172, 201, 229, 257, 284, 311]
# y_coords =  [2.2, 2.0, 1.8, 1.6, 1.4, 1.2, 1.0, 0.8, 0.6, 0.4, 0.2]
# y_axis = generate_arb_y_axis_map(y_indices, y_coords, img_vals)

# Plot to verify axis values are correct

In [ ]:
fig,ax1 = subplots(1,1)
img_min = np.min(img_vals)
img_max = np.max(img_vals)
ax1.imshow(img_vals,
           norm=matplotlib.colors.Normalize(vmin=img_min, vmax=img_max),
           # norm=matplotlib.colors.LogNorm(vmin=np.percentile(img_array, 50), vmax=np.percentile(img_array, 99.9)),
           cmap='turbo',
           extent=(np.min(x_axis), np.max(x_axis), np.min(y_axis), np.max(y_axis)))


# # For plotting non-linearly spaced x and y axes with pcolormesh:
# fig,ax1 = subplots(1,1)
# X, Y = np.meshgrid(x_axis, y_axis)
# ax1.pcolormesh(
#     X, Y,
#     img_vals,
#     norm=matplotlib.colors.LogNorm(vmin=np.percentile(img_vals, 50), vmax=np.percentile(img_vals, 99.9)),
#     cmap='terrain'
# )
# ax1.set(aspect='equal')

# Crop image

In [ ]:
x_bounds = (0,2)
y_bounds = (0,1.85)
crop_img, crop_x_axis, crop_y_axis = crop_image(img_vals, x_bounds, y_bounds, x_axis, y_axis)
fig,ax1 = subplots(1,1)
img_min = np.min(crop_img)
img_max = np.max(crop_img)
ax1.imshow(crop_img,
           norm=matplotlib.colors.Normalize(vmin=img_min, vmax=img_max),
           # norm=matplotlib.colors.LogNorm(vmin=np.percentile(img_array, 50), vmax=np.percentile(img_array, 99.9)),
           cmap='turbo',
           extent=(np.min(crop_x_axis), np.max(crop_x_axis), np.min(crop_y_axis), np.max(crop_y_axis)))

# Use pcolormesh instead of imshow for non-linear axes
# X, Y = np.meshgrid(crop_x_axis, crop_y_axis)
# ax1.pcolormesh(
#     X, Y,
#     crop_img,
#     norm=matplotlib.colors.LogNorm(vmin=np.percentile(img_vals, 50), vmax=np.percentile(img_vals, 99.9)),
#     cmap='terrain'
# )
# ax1.set(aspect='equal')

# Save Image and axes

In [ ]:
save_dirr = f'{cwd}/outputs'
if not os.path.exists(save_dirr):
    os.mkdir(save_dirr)
img_name = 'F2_digitized'
np.save(f'{save_dirr}/{img_name}_img.npy', crop_img)
np.save(f'{save_dirr}/{img_name}_xaxis.npy', crop_x_axis)
np.save(f'{save_dirr}/{img_name}_yaxis.npy', crop_y_axis)